In [1]:
import tensorflow as tf
import os
import pandas as pd
from tqdm import tqdm
# import pdb
# pdb.set_trace()


2023-01-25 22:59:27.082732: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
##data
train_data_root = "/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data"
data_ds = tf.data.experimental.make_csv_dataset(
            os.path.join(train_data_root, "train-dataset.csv"),
            batch_size=64,
            label_name='label',
            # na_value="0",
            num_epochs=1,
            shuffle=True,
            ignore_errors=True)
#验证集
val_ds = data_ds.take(1000)
#训练集1
train_ds = data_ds.skip(1000)

#测试集
test_ds = tf.data.experimental.make_csv_dataset(
            os.path.join(train_data_root, "test-dataset.csv"),
            batch_size=64,
            # label_name='label',
            # na_value="0",
            num_epochs=1,
            shuffle=True,
            ignore_errors=True)



2023-01-25 22:59:32.429784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 22:59:33.005664: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22350 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:52:00.0, compute capability: 8.6


In [4]:
data_path = '/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data'
data = pd.read_csv(os.path.join(data_path,'train-dataset.csv'))
test_data = pd.read_csv(os.path.join(data_path,'test-dataset.csv'))

In [5]:
all_data = pd.concat([data,test_data])

In [ ]:
#热门用户
#热门商品

## 


In [6]:
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder

In [7]:
#object特征处理：suv
enc = LabelEncoder()
all_data['suv']  = enc.fit_transform(all_data.suv)

In [8]:
#时间特征处理
all_data['date'] = pd.to_datetime(all_data['logTs'], unit='ms') 
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['day'] = all_data['date'].dt.day

In [9]:
all_data.dtypes

sampleId               float64
label                  float64
pvId                    object
suv                      int64
itemId                   int64
userSeq                 object
logTs                    int64
operator                 int64
browserType              int64
deviceType               int64
osType                   int64
province                 int64
city                     int64
testSampleId           float64
date            datetime64[ns]
year                     int64
month                    int64
day                      int64
dtype: object

In [11]:
##疑问如果测试集新来一个item再embedding里面没有怎么办？

In [12]:
#数据处理


In [10]:
##config
EMBEDDING_SIZE = 10
HIDDEN_SIZE = 128

# 1.制作inputs

In [11]:
inputs = {
    'suv' : tf.keras.layers.Input(name='suv', shape=(), dtype='int32'),
    'operator' : tf.keras.layers.Input(name='operator', shape=(), dtype='int32'),
    'browserType' : tf.keras.layers.Input(name='browserType', shape=(), dtype='int32'),
    'deviceType' : tf.keras.layers.Input(name='deviceType', shape=(), dtype='int32'),
    'osType' : tf.keras.layers.Input(name='osType', shape=(), dtype='int32'),
    'province' : tf.keras.layers.Input(name='province', shape=(), dtype='int32'),
    'itemId' : tf.keras.layers.Input(name='itemId', shape=(), dtype='int32'),
    'city' : tf.keras.layers.Input(name='city', shape=(), dtype='int32'),
    
}


In [12]:
inputs

{'suv': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'suv')>,
 'operator': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'operator')>,
 'browserType': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'browserType')>,
 'deviceType': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'deviceType')>,
 'osType': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'osType')>,
 'province': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'province')>,
 'itemId': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'itemId')>,
 'city': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'city')>}

# 2.制作feature_columns

In [13]:
data.columns

Index(['sampleId', 'label', 'pvId', 'suv', 'itemId', 'userSeq', 'logTs',
       'operator', 'browserType', 'deviceType', 'osType', 'province', 'city'],
      dtype='object')

In [25]:
all_data.head()

,sampleId,label,pvId,suv,itemId,userSeq,logTs,operator,browserType,deviceType,osType,province,city,testSampleId,date,year,month,day
0,1.0,0.0,1420332726637uyIZR2O,14518,10092752,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,2022-01-01 12:24:50.894,2022,1,1
1,2.0,0.0,1420332726637uyIZR2O,14518,10085565,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,2022-01-01 12:24:50.894,2022,1,1
2,3.0,0.0,1420332726637uyIZR2O,14518,10105937,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,2022-01-01 12:24:50.894,2022,1,1
3,4.0,0.0,1420332726637uyIZR2O,14518,10082274,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,2022-01-01 12:24:50.894,2022,1,1
4,5.0,0.0,1420332726637uyIZR2O,14518,10083446,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,2022-01-01 12:24:50.894,2022,1,1


In [29]:
all_data.suv.nunique()

354976

## 对于city和浏览器类型，其实他们本质上就是字符串，统计他们的nunique就可以代表hash_bucket_nums
## 而对于suv和itemId、统计他们的nunique
logTs转化为天做embedding

In [30]:
feature_columns = {}
feature_columns['suv'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('suv',700000,tf.int64,default_value=-1),EMBEDDING_SIZE)
feature_columns['itemId'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('itemId',6000,tf.int64),EMBEDDING_SIZE)
feature_columns['operator'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('operator',900,tf.int64),EMBEDDING_SIZE)
feature_columns['browserType'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('browserType',60,tf.int64) ,EMBEDDING_SIZE)
feature_columns['deviceType'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('deviceType',10,tf.int64) ,EMBEDDING_SIZE)
feature_columns['osType'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('osType',20,tf.int64) ,EMBEDDING_SIZE)
feature_columns['province'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('province',80,tf.int64) ,EMBEDDING_SIZE)
feature_columns['city'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('city',800,tf.int64) ,EMBEDDING_SIZE)

In [31]:
feature_columns

{'suv': EmbeddingColumn(categorical_column=HashedCategoricalColumn(key='suv', hash_bucket_size=700000, dtype=tf.int64), dimension=10, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fa3f9a252b0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True, use_safe_embedding_lookup=True),
 'itemId': EmbeddingColumn(categorical_column=HashedCategoricalColumn(key='itemId', hash_bucket_size=6000, dtype=tf.int64), dimension=10, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fa3f9a25310>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True, use_safe_embedding_lookup=True),
 'operator': EmbeddingColumn(categorical_column=HashedCategoricalColumn(key='operator', hash_bucket_size=900, dtype=tf.int64), dimension=10, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fa3f9a252e0>, ckpt_to_load_from=None, tensor_name_in_ckpt=Non

In [32]:
user_col = [feature_columns['suv']]
user_features = tf.keras.layers.DenseFeatures(user_col)(inputs)

In [33]:
user_features

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_features')>

In [34]:
context_col = [feature_columns['operator'], 
               feature_columns['browserType'],
               feature_columns[ 'deviceType'], 
               feature_columns['osType'],
               feature_columns['province'],
               feature_columns['city']]
context_features = tf.keras.layers.DenseFeatures(context_col)(inputs)

In [35]:
context_features

<KerasTensor: shape=(None, 60) dtype=float32 (created by layer 'dense_features_1')>

In [36]:
item_col = [feature_columns['itemId']]
item_features = tf.keras.layers.DenseFeatures(item_col)(inputs)

In [37]:
item_features

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_features_2')>

In [38]:
all_features = tf.concat([item_features,context_features],axis=-1)

In [39]:
all_features

<KerasTensor: shape=(None, 70) dtype=float32 (created by layer 'tf.concat')>

In [40]:
#model
outputs = tf.keras.layers.Dense(1024,activation='relu')(all_features)
outputs = tf.keras.layers.Dense(256,activation='relu')(outputs)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(outputs)
model = tf.keras.Model(inputs, outputs)

In [41]:
METRICS = [
      # tf.keras.metrics.TruePositives(name='tp'),
      # tf.keras.metrics.FalsePositives(name='fp'),
      # tf.keras.metrics.TrueNegatives(name='tn'),
      # tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      # tf.keras.metrics.Precision(name='precision'),
      # tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [42]:
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=METRICS)

#train
history = model.fit(train_ds, epochs=1)

/root/miniconda3/lib/python3.8/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['sampleId', 'pvId', 'userSeq', 'logTs'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


     11/Unknown - 5s 12ms/step - loss: 0.7145 - accuracy: 0.3224 - auc: 0.5048

2023-01-25 23:10:23.468104: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


65154/65154 [==============================] - 765s 12ms/step - loss: 0.4847 - accuracy: 0.8074 - auc: 0.5636


In [43]:
test_loss, test_accuracy,test_auc = model.evaluate(val_ds)

1000/1000 [==============================] - 10s 9ms/step - loss: 0.5287 - accuracy: 0.7877 - auc: 0.5115


In [44]:
y_pred = model.predict(test_ds,batch_size = 1)


/root/miniconda3/lib/python3.8/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['testSampleId', 'pvId', 'userSeq', 'logTs'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


22223/22223 [==============================] - 180s 8ms/step


In [49]:
y_pred

array([[0.14771816],
       [0.24361049],
       [0.153635  ],
       ...,
       [0.17153673],
       [0.18775031],
       [0.13141042]], dtype=float32)